# Tokyo Sushi Map

## Introduction:
My girlfriend and I will be traveling to Tokyo this summer and being the sushi lovers we are, we want convenient way to stake out the best ares of Tokyo to eat delicious lunch and dinners.

This code uses Data obtained from the Yelp API to map out the locations of the best sushi shops in Tokyo. 


In [1]:
# imports
import json
import pandas as pd
from yelpapi import YelpAPI
import os, json, math, time
from tqdm.notebook import tqdm_notebook

from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# fetching API credentials
with open('/Users/jacksonmuehlbauer/.secret/yelp_api.json') as f:
    login = json.load(f)

login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# make yelp_api variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [4]:
# search conditions
LOCATION = 'Tokyo, Japan'
TERM = 'Sushi'

In [11]:
# Instantiate json file name to store Yelp data
JSON_FILE = 'Data/tokyo_sushi.json'

In [9]:
# function to create JSON file
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [13]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists = True)

## Load previous results and use len of results for offset

with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)
    
# set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

# use out yelp_api variable's search_query method to perform our API call

results = yelp_api.search_query(location = LOCATION, term = TERM, offset = n_results)

## How many results total?
total_results = results['total']

## How many did we got the details for?
results_per_page = len(results['businesses'])

# Use math.ceil to round up for the total number of pages of results
n_pages = math.ceil((results['total']-n_results)/results_per_page)
n_pages

[!] Data/tokyo_sushi.json already exists. Deleting previous file...
[i] Data/tokyo_sushi.json not found. Saving empty list to new file.
- 0 previous results found.


85

In [15]:
# Fetching data from Yelp API
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/85 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [17]:
# load json file
df = pd.read_json(JSON_FILE)
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,99Sj_zOKkFWglZ3w0_2VDw,すしざんまい-本店-中央区,SUSHIZANMAI Honten,https://s3-media2.fl.yelpcdn.com/bphoto/awbiuv...,False,https://www.yelp.com/biz/%E3%81%99%E3%81%97%E3...,162,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 35.665917, 'longitude': 139.770608}",[],￥￥,"{'address1': '築地4-11-9', 'address2': '', 'addr...",+81335411117,+81 3-3541-1117,2152.458425
1,7r7GNKYRS3AwxzNE94CNLA,築地-斉藤水産-中央区,Tsukiji Saito's Fish Market,https://s3-media1.fl.yelpcdn.com/bphoto/tJmuZC...,False,https://www.yelp.com/biz/%E7%AF%89%E5%9C%B0-%E...,49,"[{'alias': 'seafoodmarkets', 'title': 'Seafood...",5.0,"{'latitude': 35.665527, 'longitude': 139.77049...",[],￥￥,"{'address1': '築地4-10-5', 'address2': None, 'ad...",+81335412314,+81 3-3541-2314,2193.611785
2,tVnM-RBSOaaeVeLgyt1j6w,すしざんまい-浅草雷門店-台東区,Sushizanmai Asakusakaminarimon,https://s3-media4.fl.yelpcdn.com/bphoto/co2q0n...,False,https://www.yelp.com/biz/%E3%81%99%E3%81%97%E3...,34,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.5,"{'latitude': 35.7112256125179, 'longitude': 13...",[],￥,"{'address1': '浅草1丁目3-6', 'address2': None, 'ad...",+81358261375,+81 3-5826-1375,3909.566155
3,jk3HWTcRje8_bqI6z86JAA,日向丸-浅草本店-台東区,Hinatomaru Asakusa,https://s3-media1.fl.yelpcdn.com/bphoto/V04Jbu...,False,https://www.yelp.com/biz/%E6%97%A5%E5%90%91%E4...,48,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.5,"{'latitude': 35.711847, 'longitude': 139.795839}",[],￥,"{'address1': '浅草1-20-10', 'address2': '', 'add...",+81358064222,+81 3-5806-4222,3978.818528
4,B-mK2UEmHN9TRDDtzvMKrg,魚がし日本一-新宿西口店-新宿区,Uogashi Nihon Ichi Shinjuku Nishiguchi,https://s3-media3.fl.yelpcdn.com/bphoto/IHxsk7...,False,https://www.yelp.com/biz/%E9%AD%9A%E3%81%8C%E3...,119,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 35.6899698, 'longitude': 139.6972...",[],￥,"{'address1': '西新宿1-12', 'address2': '河西ビル1F', ...",+81333491739,+81 3-3349-1739,6292.266128


In [18]:
# Exploring data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1000 non-null   object 
 1   alias          1000 non-null   object 
 2   name           1000 non-null   object 
 3   image_url      1000 non-null   object 
 4   is_closed      1000 non-null   bool   
 5   url            1000 non-null   object 
 6   review_count   1000 non-null   int64  
 7   categories     1000 non-null   object 
 8   rating         1000 non-null   float64
 9   coordinates    1000 non-null   object 
 10  transactions   1000 non-null   object 
 11  price          474 non-null    object 
 12  location       1000 non-null   object 
 13  phone          1000 non-null   object 
 14  display_phone  1000 non-null   object 
 15  distance       1000 non-null   float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 118.3+ KB


In [27]:
# Converting coordinates to usable features
df['latitude'] = [x['latitude'] for x in df['coordinates']]
df['longitude'] = [x['longitude'] for x in df['coordinates']]
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,99Sj_zOKkFWglZ3w0_2VDw,すしざんまい-本店-中央区,SUSHIZANMAI Honten,https://s3-media2.fl.yelpcdn.com/bphoto/awbiuv...,False,https://www.yelp.com/biz/%E3%81%99%E3%81%97%E3...,162,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 35.665917, 'longitude': 139.770608}",[],￥￥,"{'address1': '築地4-11-9', 'address2': '', 'addr...",+81335411117,+81 3-3541-1117,2152.458425,35.665917,139.770608
1,7r7GNKYRS3AwxzNE94CNLA,築地-斉藤水産-中央区,Tsukiji Saito's Fish Market,https://s3-media1.fl.yelpcdn.com/bphoto/tJmuZC...,False,https://www.yelp.com/biz/%E7%AF%89%E5%9C%B0-%E...,49,"[{'alias': 'seafoodmarkets', 'title': 'Seafood...",5.0,"{'latitude': 35.665527, 'longitude': 139.77049...",[],￥￥,"{'address1': '築地4-10-5', 'address2': None, 'ad...",+81335412314,+81 3-3541-2314,2193.611785,35.665527,139.770494
2,tVnM-RBSOaaeVeLgyt1j6w,すしざんまい-浅草雷門店-台東区,Sushizanmai Asakusakaminarimon,https://s3-media4.fl.yelpcdn.com/bphoto/co2q0n...,False,https://www.yelp.com/biz/%E3%81%99%E3%81%97%E3...,34,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.5,"{'latitude': 35.7112256125179, 'longitude': 13...",[],￥,"{'address1': '浅草1丁目3-6', 'address2': None, 'ad...",+81358261375,+81 3-5826-1375,3909.566155,35.711226,139.795546
3,jk3HWTcRje8_bqI6z86JAA,日向丸-浅草本店-台東区,Hinatomaru Asakusa,https://s3-media1.fl.yelpcdn.com/bphoto/V04Jbu...,False,https://www.yelp.com/biz/%E6%97%A5%E5%90%91%E4...,48,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.5,"{'latitude': 35.711847, 'longitude': 139.795839}",[],￥,"{'address1': '浅草1-20-10', 'address2': '', 'add...",+81358064222,+81 3-5806-4222,3978.818528,35.711847,139.795839
4,B-mK2UEmHN9TRDDtzvMKrg,魚がし日本一-新宿西口店-新宿区,Uogashi Nihon Ichi Shinjuku Nishiguchi,https://s3-media3.fl.yelpcdn.com/bphoto/IHxsk7...,False,https://www.yelp.com/biz/%E9%AD%9A%E3%81%8C%E3...,119,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 35.6899698, 'longitude': 139.6972...",[],￥,"{'address1': '西新宿1-12', 'address2': '河西ビル1F', ...",+81333491739,+81 3-3349-1739,6292.266128,35.689970,139.697290


In [40]:
# Plot on map

fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", color="rating", size="review_count",
                  color_continuous_scale=px.colors.diverging.Picnic, size_max=100, zoom=10,
                  mapbox_style="carto-positron", hover_data=['name', 'review_count', 'rating'])
fig.show()

In [41]:
fig.write_html('full_html.html')
fig.write_html('small_html.html', full_html = False, include_plotlyjs = 'cdn')